In [1]:
%matplotlib inline

In [2]:
# dependencies

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine

import datetime

# postgres pull

db_password = "postgres"
db_user = "postgres"
db_name = "Youtube P3"
endpoint = 'youtube.cb1bticre0py.us-east-1.rds.amazonaws.com'

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

og_data = pd.read_sql('SELECT * FROM all_data;', con = engine)

In [3]:
# start here if you need to re-run instead of pulling from postgres in above cell

all_data = og_data.copy()

In [4]:
# BEGIN INITIAL CLEANUP ON ENTIRE DATASET
# formats, dropping nulls, etc

# turn user engagement nulls to medians
all_data[['view_count','likes','dislikes','comment_count']] = all_data[['view_count','likes','dislikes','comment_count'
    ]].replace(0,all_data[['view_count', 'likes','dislikes', 'comment_count'
    ]].median())

# remove video_id's == '#NAME?'   (about 1,300 records)
all_data.drop(all_data[all_data.video_id == '#NAME?'].index, inplace=True)

# changed publish_date & trending_date to datetime format
all_data[["publishedat", "trending_date"]] = all_data[["publishedat", "trending_date"]].apply(pd.to_datetime)

# sort (important for the drop_duplicates later on)
all_data.sort_values('trending_date', inplace = True)

# clean given column names
all_data.rename(columns={'publishedat': 'publish_date', 
                         'channelid': 'channel_id', 
                         'channeltitle': 'channel_title', 
                         'likes':'likes_count', 
                         'dislikes': 'dislikes_count', 
                         'cat_codes': 'category'
                        }, inplace=True)

# all_data.reset_index(inplace=True)
# print(all_data.shape)
# print('')
print(all_data.columns)
# print('')
# print(all_data.dtypes)
# print(' ')
# all_data.head(2)

Index(['video_id', 'publish_date', 'channel_id', 'channel_title',
       'trending_date', 'view_count', 'likes_count', 'dislikes_count',
       'comment_count', 'thumbnail_link', 'country', 'category'],
      dtype='object')


In [5]:
## BEGIN FEATURE ENGINEERING
# datetime columns

# add column 'combined_trend_days' == cumulative days a video appears on each country trending list

temp_df = all_data.groupby('video_id')['trending_date'].count().reset_index()
temp_df.rename({'trending_date':'combined_trend_days'}, axis = 'columns', inplace = True)
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')


# add columns 'day_of_week' and 'number_of_day'  == day of week video first appeared on trending list (i.e. Tuesday)

all_data['publish_day'] = all_data['publish_date'].dt.day_name()
all_data['publish_day_num'] = all_data['publish_date'].dt.day_of_week


# add column 'publish_to_trend'  ==  count of days between posting to youtube and appearing on trending list

all_data['publish_to_trend'] = round((all_data['trending_date'] - all_data['publish_date']).dt.days.astype(float),2)


# add columns user-interactions ratios  ==  likes per view & comments per view

all_data['likes_ratio'] = round(all_data['likes_count'] / all_data['view_count']*100,2)
all_data['comments_ratio'] = round(all_data['comment_count'] / all_data['view_count']*100,2)


# add column 'trend_days'  ==  count of days between first and last days on trending list

##### temporary column for last trending date
temp_df = all_data.groupby('video_id')['trending_date'].max()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'trending_date_y':'trending_date_max'}, axis = 'columns', inplace = True)
all_data.rename({'trending_date_x':'trending_date'}, axis = 'columns', inplace = True)
##### temporary column for first trending date
temp_df = all_data.groupby('video_id')['trending_date'].min()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'trending_date_y':'trending_date_min'}, axis = 'columns', inplace = True)
all_data.rename({'trending_date_x':'trending_date'}, axis = 'columns', inplace = True)
##### subtract min from max to get final column
all_data['trend_days'] = ((all_data['trending_date'] - all_data['trending_date_min']).dt.days.astype(float))+1

# label encode category and country columns

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
all_data['category_e'] = (encoder.fit_transform(all_data.category.values)).astype(float)

In [6]:
# all_data.dtypes

In [7]:
# ADD PRE TRENDING COLUMNS for likes, dislikes, comments, and views

# KEY:  pt == pre-trending    wt == while trending

# add column 'pt_likes' and 'pt_likes_rate'

temp_df = all_data.groupby('video_id')['likes_count'].min()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'likes_count_y':'pt_likes'}, axis = 'columns', inplace = True)
all_data.rename({'likes_count_x':'likes'}, axis = 'columns', inplace = True)
all_data['pt_likes_rate'] = round(all_data['pt_likes'] / all_data['publish_to_trend'],2)

# add column 'pt_dislikes' and 'pt_dislikes_rate'

temp_df = all_data.groupby('video_id')['dislikes_count'].min()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'dislikes_count_y':'pt_dislikes'}, axis = 'columns', inplace = True)
all_data.rename({'dislikes_count_x':'dislikes'}, axis = 'columns', inplace = True)
all_data['pt_dislikes_rate'] = round(all_data['pt_dislikes'] / all_data['publish_to_trend'],2)

# add column 'pt_comments' & 'pt_comments_rate'

temp_df = all_data.groupby('video_id')['comment_count'].min()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'comment_count_y':'pt_comments'}, axis = 'columns', inplace = True)
all_data.rename({'comment_count_x':'comments'}, axis = 'columns', inplace = True)
all_data['pt_comments_rate'] = round(all_data['pt_comments'] / all_data['publish_to_trend'],2)

# add column 'pt_views' & 'pt_views_rate'

temp_df = all_data.groupby('video_id')['view_count'].min()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'view_count_y':'pt_views'}, axis = 'columns', inplace = True)
all_data.rename({'view_count_x':'views'}, axis = 'columns', inplace = True)
all_data['pt_views_rate'] = all_data['pt_views'] / all_data['publish_to_trend'].astype(float)

In [8]:
# ADD WHILE TRENDING COLUMNS for likes, dislikes, comments, and views

# add column 'wt_likes'

##### temporary column for first trending day likes
temp_df = all_data.groupby('video_id')['likes'].min()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'likes_y':'min_likes_count'}, axis = 'columns', inplace = True)
all_data.rename({'likes_x':'likes'}, axis = 'columns', inplace = True)
##### temporary column for last trending day likes
temp_df = all_data.groupby('video_id')['likes'].max()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'likes_y':'max_likes_count'}, axis = 'columns', inplace = True)
all_data.rename({'likes_x':'likes'}, axis = 'columns', inplace = True)
##### subtract min from max to get final column
all_data['wt_likes'] = all_data['max_likes_count'] - all_data['min_likes_count']
all_data.drop(columns = ['min_likes_count', 'max_likes_count'], inplace=True)

# add column 'wt_likes_rate'

#all_data["tempCol"] = all_data["trend_days"] / datetime.timedelta(days=1)
all_data['wt_likes_rate'] = round(all_data['wt_likes'] / all_data['trend_days'],2)
#all_data.drop(columns='tempCol')

# add column 'wt_dislikes'

##### temporary column for min dislikes count
temp_df = all_data.groupby('video_id')['dislikes'].min()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'dislikes_y':'min_dislikes_count'}, axis = 'columns', inplace = True)
all_data.rename({'dislikes_x':'dislikes'}, axis = 'columns', inplace = True)
##### temporary column for max dislikes count
temp_df = all_data.groupby('video_id')['dislikes'].max()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'dislikes_y':'max_dislikes_count'}, axis = 'columns', inplace = True)
all_data.rename({'dislikes_x':'dislikes'}, axis = 'columns', inplace = True)
##### subtract min from max to get final column
all_data['wt_dislikes'] = all_data['max_dislikes_count'] - all_data['min_dislikes_count']
all_data.drop(columns = ['min_dislikes_count', 'max_dislikes_count'], inplace=True)

# add column 'wt_dislikes_rate'

#all_data["tempCol"] = all_data["trend_days"] / datetime.timedelta(days=1)
all_data['wt_dislikes_rate'] = round(all_data['wt_dislikes'] / all_data['trend_days'],2)
#all_data.drop(columns='tempCol')

# add column 'wt_comments'

##### temporary column for min comment count
temp_df = all_data.groupby('video_id')['comments'].min()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'comments_y':'min_comment_count'}, axis = 'columns', inplace = True)
all_data.rename({'comments_x':'comments'}, axis = 'columns', inplace = True)
##### temporary column for max comment count
temp_df = all_data.groupby('video_id')['comments'].max()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'comments_y':'max_comment_count'}, axis = 'columns', inplace = True)
all_data.rename({'comments_x':'comments'}, axis = 'columns', inplace = True)
##### subtract min from max to get final column
all_data['wt_comments'] = all_data['max_comment_count'] - all_data['min_comment_count']
all_data.drop(columns = ['min_comment_count', 'max_comment_count'], inplace=True)

# add column 'wt_comments_rate'

#all_data["tempCol"] = all_data["trend_days"] / datetime.timedelta(days=1)
all_data['wt_comments_rate'] = round(all_data['wt_comments'] / all_data['trend_days'],2)
#all_data.drop(columns='tempCol', inplace=True)

# # add column 'wt_views'

##### temporary column for min view_count
temp_df = all_data.groupby('video_id')['views'].min()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'views_y':'min_view_count'}, axis = 'columns', inplace = True)
all_data.rename({'views_x':'views'}, axis = 'columns', inplace = True)
##### temporary column for max view_count
temp_df = all_data.groupby('video_id')['views'].max()
all_data = all_data.merge(temp_df, on = 'video_id', how = 'inner')
all_data.rename({'views_y':'max_view_count'}, axis = 'columns', inplace = True)
all_data.rename({'views_x':'views'}, axis = 'columns', inplace = True)
##### subtract min from max to get final column
all_data['wt_views'] = all_data['max_view_count'] - all_data['min_view_count']
all_data.drop(columns = ['min_view_count', 'max_view_count'], inplace=True)

# add column 'wt_views_rate'

#all_data["tempCol"] = all_data["trend_days"] / datetime.timedelta(days=1)
all_data['wt_views_rate'] = round(all_data['wt_views'] / all_data['trend_days'],2)
#all_data.drop(columns='tempCol', inplace=True)

In [9]:
# re-organize columns

all_data = all_data[[
                     'video_id', 
                     'category',
                     'category_e',
                     'country',
                     'publish_date',
                     'trending_date',
                     'publish_to_trend',
                     'publish_day',
                     'publish_day_num',
                     'combined_trend_days', 
                     'trend_days', 
                     'views',
                     'pt_views',
                     'wt_views',
                     'pt_views_rate',
                     'wt_views_rate',
                     'likes',
                     'pt_likes',
                     'wt_likes',
                     'pt_likes_rate',
                     'wt_likes_rate',
                     'likes_ratio',
                     'dislikes',
                     'pt_dislikes',
                     'wt_dislikes',
                     'pt_dislikes_rate',
                     'wt_dislikes_rate',
                     'comments',
                     'pt_comments',
                     'wt_comments',
                     'pt_comments_rate',
                     'wt_comments_rate',
                     'comments_ratio']]

In [10]:
# UNSCALED all_data 

all_data.reset_index()
print(all_data.shape)
# print('')
# print(all_data.columns)
# print('')
# print(all_data.dtypes)
# print(' ')
all_data.head(2)

(1298907, 33)


,video_id,category,category_e,country,publish_date,trending_date,publish_to_trend,publish_day,publish_day_num,combined_trend_days,...,pt_dislikes,wt_dislikes,pt_dislikes_rate,wt_dislikes_rate,comments,pt_comments,wt_comments,pt_comments_rate,wt_comments_rate,comments_ratio
0,e_Z6upbeYuc,Entertainment,3.0,Russia,2020-08-12,2020-08-12,0.0,Wednesday,2,4,...,29927.0,34070.0,inf,34070.0,63557.0,63557.0,53970.0,inf,53970.0,1.23
1,e_Z6upbeYuc,Entertainment,3.0,Russia,2020-08-12,2020-08-13,1.0,Wednesday,2,4,...,29927.0,34070.0,29927.0,17035.0,92009.0,63557.0,53970.0,63557.0,26985.0,0.96


In [11]:
## final_all_data SENT TO POSTRES

from sqlalchemy import create_engine

all_data.to_sql('final_all_data', engine)

In [14]:
# UNSCALED unique 

# drop duplicates video_id's leaving us with the row corresponding to the videos last trending date

unique = all_data.drop_duplicates(subset='video_id', keep='last').reset_index(drop=True)
print(unique.shape)
print(unique.columns)
unique.head(2)

(241112, 33)
Index(['video_id', 'category', 'category_e', 'country', 'publish_date',
       'trending_date', 'publish_to_trend', 'publish_day', 'publish_day_num',
       'combined_trend_days', 'trend_days', 'views', 'pt_views', 'wt_views',
       'pt_views_rate', 'wt_views_rate', 'likes', 'pt_likes', 'wt_likes',
       'pt_likes_rate', 'wt_likes_rate', 'likes_ratio', 'dislikes',
       'pt_dislikes', 'wt_dislikes', 'pt_dislikes_rate', 'wt_dislikes_rate',
       'comments', 'pt_comments', 'wt_comments', 'pt_comments_rate',
       'wt_comments_rate', 'comments_ratio'],
      dtype='object')


,video_id,category,category_e,country,publish_date,trending_date,publish_to_trend,publish_day,publish_day_num,combined_trend_days,...,pt_dislikes,wt_dislikes,pt_dislikes_rate,wt_dislikes_rate,comments,pt_comments,wt_comments,pt_comments_rate,wt_comments_rate,comments_ratio
0,e_Z6upbeYuc,Entertainment,3.0,Russia,2020-08-12,2020-08-15,3.0,Wednesday,2,4,...,29927.0,34070.0,9975.67,8517.5,117527.0,63557.0,53970.0,21185.67,13492.5,0.92
1,Ou5ksu78utU,News & Politics,8.0,Russia,2020-08-10,2020-08-12,2.0,Monday,0,1,...,394.0,0.0,197.00,0.0,1226.0,1226.0,0.0,613.00,0.0,0.46


In [16]:
## final_unique SENT TO POSTRES

unique.to_sql('final_unique', engine)